In [32]:
import pandas as pd
import sqlite3
# import psycopg2

In [3]:
df = pd.read_csv('Analysis911/911_Police_Calls_for_Service.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df['CallDateTime'] = pd.to_datetime(df['CallDateTime'])
df = df.dropna(subset=['Census_Tracts'])
df = df.dropna(subset=['PoliceDistrict'])
df = df.dropna(subset=['Priority'])
df['Count'] = 1

def cen_tract(tract_string):
    tract = float(str(tract_string).split()[2])
    return tract

df.Census_Tracts = df.apply(lambda x: cen_tract(x['Census_Tracts']),axis=1)
df.info()

In [14]:
df = df.rename(columns ={'2010 Census Neighborhoods':'2010_Census_Neighborhoods',
                        '2010 Census Wards Precincts':'2010_Census_Wards_Precincts',
                        'Zip Codes':'Zip_Codes'})

In [38]:
dfRecordID = df[['RecordID','CallNumber','CallDateTime']].set_index('RecordID')
dfLocation = df[['RecordID','Census_Tracts','PoliceDistrict','District','IncidentLocation',
                 'ZipCode','Neighborhood',
                 'PolicePost','CouncilDistrict','SheriffDistricts',
                 'Community_Statistical_Areas',
                 'VRIZones','Location'
                 ]].set_index('RecordID')
dfDetails = df[['RecordID','Priority','Description']].set_index('RecordID')

In [39]:
dfRecordID.to_csv('RecordID.csv')
dfLocation.to_csv('LocationID.csv')
dfDetails.to_csv('DetailsID.csv')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7745304 entries, 0 to 7823489
Data columns (total 21 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   RecordID                     int64         
 1   CallNumber                   object        
 2   CallDateTime                 datetime64[ns]
 3   Priority                     object        
 4   District                     object        
 5   Description                  object        
 6   IncidentLocation             object        
 7   ZipCode                      float64       
 8   Neighborhood                 object        
 9   PoliceDistrict               object        
 10  PolicePost                   float64       
 11  CouncilDistrict              float64       
 12  SheriffDistricts             object        
 13  Community_Statistical_Areas  object        
 14  Census_Tracts                float64       
 15  VRIZones                     object        
 16  

In [55]:
connection = sqlite3.connect('C:/Users/tsen6/Desktop/Python/Baltimore_911_01.db')
cursor = connection.cursor()

In [56]:
# #creating table passengers
# cursor.execute("""CREATE TABLE RecordID(
# RecordID SERIAL PRIMARY KEY,
# CallNumber integer,
# CallDateTime datetime)""")

In [57]:
# cursor.execute("""CREATE TABLE LocationID(
# RecordID SERIAL PRIMARY KEY,
# Census_Tracts float,
# PoliceDistrict text,
# District text,
# IncidentLocation text,
# ZipCode integer,
# Neighborhood text,
# PolicePost text,
# CouncilDistrict text,
# SheriffDistricts text,
# Community_Statistical_Areas text,
# VRIZones text,
# Location text)""")

# cursor.execute("""CREATE TABLE DetailsID(
# RecordID SERIAL PRIMARY KEY,
# Priority text,
# Description text)""")

# connection.commit()

In [45]:
dfDetails.to_sql('DetailsID', con=connection, if_exists='append')  

dfLocation.to_sql('LocationID', con=connection, if_exists='append')  

dfRecordID.to_sql('RecordID', con=connection, if_exists='append')  

connection.commit()

In [50]:
sql = """
SELECT RecordID, CallDateTime, Priority, Census_Tracts, PoliceDistrict
FROM (SELECT r.RecordID, r.CallDateTime, d.Priority, l.Census_Tracts, l.PoliceDistrict
FROM RecordID r
INNER JOIN DetailsID d ON r.RecordID = d.RecordID
INNER JOIN LocationID l ON r.RecordID = l.RecordID
WHERE (d.Priority = 'High') OR (d.Priority = 'Medium'))
"""
pd.read_sql(sql, con=connection)

,RecordID,CallDateTime,Priority,Census_Tracts,PoliceDistrict
0,6144905,2019-04-11 08:01:00,Medium,1501.00,Western
1,7473156,2020-02-27 09:56:00,Medium,401.00,Central
2,7473387,2020-02-27 11:00:00,Medium,2505.00,Southern
3,7473302,2020-02-27 10:22:00,High,1504.00,Western
4,7473570,2020-02-27 11:33:00,High,2201.00,Southern
...,...,...,...,...,...
4065948,5841920,2013-07-30 22:18:00,Medium,2002.00,Southwestern
4065949,5841735,2013-07-30 23:34:00,Medium,2804.03,Southwestern
4065950,5842440,2013-07-30 19:23:00,Medium,1605.00,Western
4065951,5842176,2013-07-30 20:57:00,Medium,1901.00,Western
